### Setup

In [173]:
!pip install hbase-python

Defaulting to user installation because normal site-packages is not writeable


In [174]:
!pip install --upgrade pip setuptools

Defaulting to user installation because normal site-packages is not writeable


In [175]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [176]:
import happybase

In [177]:
import pandas as pd

In [178]:
import warnings
warnings.filterwarnings('ignore')

In [179]:
import findspark

findspark.init() 

### Setup connection

In [180]:
connection = happybase.Connection('localhost')

In [181]:
table = connection.table("binance_currencies")

In [182]:
table_kucoin = connection.table("kucoin_currencies")

#### Create pandas dataframe

In [183]:
df = pd.DataFrame([], columns=['currency', 'price'])

In [184]:
for t in table.scan():
    print(t)
    #df1 = pd.DataFrame([[t[0], t[1][b'price:price']]], columns=['currency', 'price'])
    #df = df.append(df1)

(b'1INCHBTC', {b'price:price': b'0.00002423'})
(b'1INCHBUSD', {b'price:price': b'0.40600000'})
(b'1INCHDOWNUSDT', {b'price:price': b'0.00614800'})
(b'1INCHUPUSDT', {b'price:price': b'0.06570300'})
(b'1INCHUSDT', {b'price:price': b'0.40600000'})
(b'AAVEBKRW', {b'price:price': b'164167.00000000'})
(b'AAVEBNB', {b'price:price': b'0.23260000'})
(b'AAVEBRL', {b'price:price': b'1792.00000000'})
(b'AAVEBTC', {b'price:price': b'0.00339700'})
(b'AAVEBUSD', {b'price:price': b'57.00000000'})
(b'AAVEDOWNUSDT', {b'price:price': b'0.00467300'})
(b'AAVEETH', {b'price:price': b'0.04700000'})
(b'AAVEUPUSDT', {b'price:price': b'0.60080000'})
(b'AAVEUSDT', {b'price:price': b'57.00000000'})
(b'ACABTC', {b'price:price': b'0.00000725'})
(b'ACABUSD', {b'price:price': b'0.12200000'})
(b'ACAUSDT', {b'price:price': b'0.12220000'})
(b'ACHBTC', {b'price:price': b'0.00000049'})
(b'ACHBUSD', {b'price:price': b'0.00810000'})
(b'ACHUSDT', {b'price:price': b'0.00811000'})
(b'ACMBTC', {b'price:price': b'0.00013970'})
(

In [185]:
df.currency = df.currency.apply(lambda x: str(x).replace('b','').replace("'", ""))
df.price = df.price.apply(lambda x: float(str(x).replace('b','').replace("'", "")))

In [186]:
df = df.reset_index(drop=True)

In [187]:
df.to_csv("binance.csv", index=False)

#### Pandas dataframe - kucoin

In [188]:
df1 = pd.DataFrame([], columns=['currency', 'buy', 'sell'])

In [189]:
list_cols = []

In [190]:
for t in table_kucoin.scan():
    print(t)
    #df2 = pd.DataFrame([[t[0], t[1][b'price:buy']], t[1][b'price:sell']], columns=['currency', 'buy', 'sell'])
    #df1 = df1.append(df2)

(b'1EARTH-USDT', {b'price:averagePrice': b'0.00532979', b'price:buy': b'0.00515998', b'price:changePrice': b'-0.00023434', b'price:changeRate': b'-0.0432', b'price:high': b'0.005689', b'price:last': b'0.00518665', b'price:low': b'0.0048465', b'price:makerCoefficient': b'2', b'price:makerFeeRate': b'0.001', b'price:sell': b'0.00518665', b'price:symbolName': b'1EARTH-USDT', b'price:takerCoefficient': b'2', b'price:takerFeeRate': b'0.001', b'price:vol': b'353483406.38224745', b'price:volValue': b'1874469.3885362533961642'})
(b'1INCH-USDT', {b'price:averagePrice': b'0.40514553', b'price:buy': b'0.39934', b'price:changePrice': b'-0.00504', b'price:changeRate': b'-0.0124', b'price:high': b'0.41408', b'price:last': b'0.39928', b'price:low': b'0.39649', b'price:makerCoefficient': b'1', b'price:makerFeeRate': b'0.001', b'price:sell': b'0.4', b'price:symbolName': b'1INCH-USDT', b'price:takerCoefficient': b'1', b'price:takerFeeRate': b'0.001', b'price:vol': b'122296.1918', b'price:volValue': b'49

In [191]:
df1

,currency,buy,sell


### Pyspark

In [192]:
from pyspark.sql import SQLContext
from pyspark import SparkContext

In [193]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [194]:
df = sqlContext.read.format('org.apache.hadoop.hbase.spark') \
    .option('hbase.table','binance_currencies') \
    .option('hbase.columns.mapping', \
            'title STRING :key, \
            author STRING price:price') \
    .load()

Py4JJavaError: An error occurred while calling o61.load.
: java.lang.ClassNotFoundException: Failed to find data source: org.apache.hadoop.hbase.spark. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:692)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:746)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:265)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:225)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at p

In [ ]:
hbaseDF= sqlContext.read.format("org.apache.spark.sql.execution.datasources.hbase").load()